介绍自定义层  
可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构  
发明一个现在在深度学习框架中还不存在的层

不带参数的层  
构造一个没有任何参数的自定义层  
下面的CenteredLayer类要从其输入中减去均值。  
要构建它，我们只需继承基础层类并实现前向传播功能。

In [2]:
import torch 
import torch.nn.functional as F
from torch import nn
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()


    def forward(self,X):
        return X-X.mean()

向该层提供一些数据，验证它是否能按预期工作

In [3]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))


tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以将层作为组件合并到更复杂的模型中。

In [4]:
net=nn.Sequential(
    nn.Linear(8,128),
    CenteredLayer()
)

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。  
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。

In [5]:
Y=net(torch.rand(4,8))
Y.mean()

tensor(4.0745e-09, grad_fn=<MeanBackward0>)

带参数的层  
继续定义具有参数的层， 这些参数可以通过训练进行调整。  
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。  
比如管理访问、初始化、共享、保存和加载模型参数。  
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。 
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。  
在此实现中，我们使用修正线性单元(relu)作为激活函数。  
该层需要输入参数：in_units和units，分别表示输入数d和输出数q。
$$\vec o_{1 \times q}=\vec{x}_{1 \times d}W_{d \times q}+\vec{b}_{1 \times q}$$

In [7]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(in_units,units))
        self.bias=nn.Parameter(torch.rand(units,))
    
    def forward(self,X):
        linear=torch.mm(X,self.weight.data)+self.bias.data
        return F.relu(linear)


实例化MyLinear类并访问其模型参数

In [8]:
linear = MyLinear(5, 3)
linear.weight


Parameter containing:
tensor([[0.2801, 0.0844, 0.5043],
        [0.1674, 0.7256, 0.4996],
        [0.8017, 0.6882, 0.4554],
        [0.8114, 0.0291, 0.0189],
        [0.4198, 0.3908, 0.6795]], requires_grad=True)

以使用自定义层直接执行前向传播计算

In [9]:
linear(torch.rand(2, 5))

tensor([[1.9057, 1.7435, 1.5225],
        [0.8720, 1.7081, 1.2995]])

还可以使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层

In [10]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))


tensor([[60.4864],
        [64.4479]])